In [ ]:
from PacTimeOrig.data import DataHandling as dh
from PacTimeOrig.data import DataProcessing as dp
from PacTimeOrig.Attractor import base as attractor
from PacTimeOrig.utils import processing as proc
from PacTimeOrig.models import NNtf,io
import keras.backend as K
import scipy

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM,GRU, Dense, Lambda,TimeDistributed,SimpleRNN,LeakyReLU
import matplotlib.pyplot as plt

import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader





In [1]:
#load datafile
datafile=dh.dataloader(sess=1)

#Get session variables
sessvars=dh.getvars(datafile)

#Get position data
positions=dh.retrievepositions(datafile,rescale = [960.00,540.00])
# positions=dh.retrievepositions(datafile,rescale = None)

kinematics=dp.computederivatives(positions, vartodiff=['selfXpos','selfYpos','prey1Xpos','prey1Ypos','prey2Xpos','prey2Ypos'], dt=1.0/60.0)
kinematics,sessvars=dh.subselect(kinematics,sessvars,trialtype='2')
kinematics=dh.dropcols(kinematics, columns_to_drop=['predXpos','predYpos','selfXaccel','selfYaccel','prey1Xaccel','prey1Yaccel','prey2Xaccel','prey2Yaccel'])



NameError: name 'dh' is not defined

Add in reward information and time vector per trial for design matrix

Create Ytrain for u(t+1)

In [ ]:
Xdsgn=kinematics
for trial in range(len(Xdsgn)):
    Xdsgn[trial]['val1'] = np.repeat(sessvars.iloc[trial].NPCvalA,len(kinematics[trial]))
    Xdsgn[trial]['val2'] = np.repeat(sessvars.iloc[trial].NPCvalB,len(kinematics[trial]))
    
#Switch reward positions so highest value is always in prey 1 slot
Xdsgn=dh.rewardalign(Xdsgn)
Xdsgn = [df[sorted(df.columns)] for df in Xdsgn]


rwds=np.sort(sessvars['NPCvalA'].unique())
for trial in range(len(Xdsgn)):
    #Make one hot columns
    for i in range(len(sessvars['NPCvalA'].unique())):
        Xdsgn[trial]['val1'+str(i+1)]=np.zeros((len(kinematics[trial]),1)).astype(int)
        Xdsgn[trial]['val2'+str(i+1)]=np.zeros((len(kinematics[trial]),1)).astype(int)
    
    Xdsgn[trial]['val1'+str((np.where(Xdsgn[trial].val1[0]==rwds)[0]+1)[0])]=1
    Xdsgn[trial]['val2'+str((np.where(Xdsgn[trial].val2[0]==rwds)[0]+1)[0])]=1
    Xdsgn[trial].drop(columns=['val1','val2'], inplace=True)



Ytrain=list()
for trial in range(len(Xdsgn)):
    Ytrain.append(np.vstack([Xdsgn[trial].selfXvel,Xdsgn[trial].selfYvel]).transpose())
    Xdsgn[trial]=Xdsgn[trial].drop(columns=['selfXvel','selfYvel'],axis=1)

Xdsgn=[df.to_numpy() for df in Xdsgn]    
# 
# # #Make from 1:end-1
Xdsgn = [arr[:-1, :] for arr in Xdsgn]
# # 
# # #Make from 2:end
# # 
Ytrain = [arr[1:, :] for arr in Ytrain]

Ytrain=proc.smoothing(Ytrain,11)



for trial in range(len(Ytrain)):
    Ytrain[trial]=np.gradient(Ytrain[trial],axis=0)

In [ ]:
batch_size = 20
val_batch_size = 20
val_data_gen = io.VariableLengthDataGenerator(Xdsgn[201:], Ytrain[201:], val_batch_size)
data_gen=io.VariableLengthDataGenerator(Xdsgn[0:200],Ytrain[0:200],batch_size)

model=NN.mod3(input_size=20,units=[128,128,32,2],celltype='simpleRNN')
# model=mod2()

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model using your data generators
model.fit(data_gen, validation_data=val_data_gen, epochs=20)



trial=35
# Predict a single trials outputs
single_trial = np.expand_dims(Xdsgn[trial], axis=1)
fig, (ax1, ax2) = plt.subplots(2)
ax1.plot(60*np.gradient(model.predict(single_trial).squeeze(),axis=0))
# ax1.plot(model.predict(single_trial).squeeze())
ax2.plot((Ytrain[trial]))

# PREDICT A LATENT DATA from a specific layer
# Create a model to output latent dynamics for a given input
latent_model = Model(inputs=model.input, outputs=model.get_layer('time_distributed').output)  

# Predict latent dynamics for the test set
latent_dynamics_output = latent_model.predict(np.expand_dims(Xdsgn[0], axis=0))  # This will give you the latent space



In [ ]:
# #Create Torch CTRNN
# model=NN.RNNNet(input_size=12, hidden_size=64,
#              output_size=2, dt=16.67,tau=50)
# # Use Adam optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# criterion = nn.MSELoss()
# 
# # Create the dataset and dataloader
# dataset = io.VariableLengthDataset(Xdsgn, Ytrain)
# dataloader=io.torchloader(dataset)
# trainmask(model,2,criterion,optimizer,dataloader,epochs=10)

In [ ]:
# import torch.nn.functional as F
# def compute_masked_loss(output, target, criterion):
#     """
#     Computes MSE loss, ignoring padded values (where target is zero).
#     """
#     mask = target != 0  # Create a mask where target is not zero (ignoring padding)
#     masked_output = output[mask]
#     masked_target = target[mask]
#     return criterion(masked_output, masked_target)
# # Training loop
# running_loss = 0
# for epoch in range(10):
#     for inputs, labels,lengths in dataloader:
#         # Move inputs and labels to float type if necessary
#         inputs = inputs.float()
#         labels = labels.float()
#         # Zero the gradient buffers
#         optimizer.zero_grad()
#         # Forward pass
#         output, _ = model(inputs)
#         # Reshape the output if necessary (depends on how the output is structured in your network)
#         output = output.view(-1, output_size)  # Ensure output has the correct shape
#         # Compute loss using a masked approach
#         loss = compute_masked_loss(output, labels.view(-1, output_size), criterion)
#         # Backward pass
#         loss.backward()
#         optimizer.step()
#         # Track loss
#         running_loss += loss.item()
#     # Logging
#     if epoch % 100 == 99:
#         avg_loss = running_loss / (len(dataloader) * 100)  # Average over all batches in 100 epochs
#         print(f'Epoch {epoch + 1}, Loss: {avg_loss:.4f}')
#         running_loss = 0

TO DO: add smoothness or sparisty losses and regularization into firing.
# skip connnections from input direct to output
# Latent layer
# multiple lstm layers
# CTRNN
# Try using u(t)

With skip connecitons

In [ ]:
from tensorflow.keras.layers import Add

# RNN/LSTM model
rnn_output = LSTM(units=128, return_sequences=True)(inputs)

# Skip connection directly from input to control output
control_output = Dense(units=2,activation='tanh')(rnn_output)
skip_connection = Dense(units=2)(inputs)
control_output = Add()([control_output, skip_connection])
# Define the model
model = Model(inputs=inputs, outputs=control_output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(data_gen, validation_data=val_data_gen, epochs=100)

# Predict a single trials outputs
single_trial = np.expand_dims(Xdsgn[42], axis=1)
plt.plot(model.predict(single_trial).squeeze())